In [128]:
from ortools.sat.python import cp_model
from time import time
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import ortools

#import local module
from utils import get_holiday, get_morn_con_day

In [137]:
# ============================= Input ============================= #

# List of staffs
staff_names = ["อ.บริบูรณ์","อ.ณัฐฐิกานต์","อ.ปริญญา","อ.ภาวิตา","อ.ธีรพล","อ.บวร","อ.ชานนท์","อ.บุญฤทธิ์","อ.กอสิน","อ.พิมพ์พรรณ","อ.กรองกาญจน์"]
num_staffs = len(staff_names)

# Select month and year to schedule & 
month = 3
year = 2023
num_days = (datetime(year, month+1, 1) - datetime(year, month, 1)).days
num_weeks = num_days // 7
date_list = [datetime(year, month, day) for day in range(1, num_days+1)]

#Get holiday list
holiday = get_holiday(year, month)

# get morning conference day
morn_con_day = get_morn_con_day(year, month)

# List of shifts
shift_names = ['service1', 'service1+', 'service2', 'service2+', 'Observe', 'Morn Con', 'EMS',  'AMD', 'off_morning', 'off_afternoon', 'off_allday']
num_shifts = len(shift_names) # 8 shifts per day, 3 off shifts


# ============================= Create model ============================= #

model = cp_model.CpModel()

# # Create shift variables.
# # shifts[(s, d, w)]: staff 's' works shift 'w' on day 'd'.
# shifts = {}
# for s in range(num_staffs):
#     for d in range(num_days):
#         for w in range(num_shifts):
#             shifts[(s, d, w)] = model.NewBoolVar('shift_s%id%iw%i' % (s, d, w))

# Create shift variables.
# shifts[(s, d, t)]: staff 's' works shift 't' on day 'd'.
shifts = {}
for s in range(num_staffs):
    for d in range(num_days):
        for t in range(num_shifts):
            shifts[(s, d, t)] = model.NewBoolVar('shift_s%id%it%i' % (s, d, t))

# ============================= Required Constraints ============================= #


# For (service1 & service2 & service1+ & service2+ & observe) Each shift is assigned to exactly one staff in that day, 
for d in range(num_days):
    if holiday[d] == False:
        for t in [0, 1, 2, 3, 4]:
            model.AddExactlyOne([shifts[(s, d, t)] for s in range(num_staffs)])


# For morning conference day, each staff must work at least 1 shift
for d in range(num_days):
    if morn_con_day[d] == True:
        model.AddExactlyOne([shifts[(s, d, 5)] for s in range(num_staffs)])

# For (EMS & AMD) Each shift is assigned to exactly one staff in that day, include holiday
for d in range(num_days):
    for t in [6, 7]:
        model.AddExactlyOne([shifts[(s, d, t)] for s in range(num_staffs)])


# Service1 & Service1+ & Observe & Morn Con & EMS & AMD can't overlap
# Service2 & Service2+ & Observe & EMS & AMD can't overlap
# Assign to off shift if no shift is assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.AddAtMostOne([shifts[(s, d, t)] for t in [0, 1, 4, 5, 6, 7]])
        model.AddAtMostOne([shifts[(s, d, t)] for t in [2, 3, 4, 6, 7]])


# assign to off_morning shift if Service1 & Service1+ & Observe & Morn Con & EMS & AMD is not assigned
# assign to off_afternoon shift if Service2 & Service2+ & Observe & EMS & AMD is not assigned
# assign to off_allday shift if no shift is assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.Add(shifts[(s, d, 8)] + sum([shifts[(s, d, t)] for t in [0, 1, 4, 5, 6, 7]]) == 1)
        model.Add(shifts[(s, d, 9)] + sum([shifts[(s, d, t)] for t in [2, 3, 4, 6, 7]]) == 1)
        model.Add(shifts[(s, d, 10)] + sum([shifts[(s, d, t)] for t in [0, 1, 2, 3, 4, 5, 6, 7]]) == 1)


# Fixed shift
# (staff, day, shift)
fixed_shift = [
    # (0, 0, 0), # service1
    # (0, 1, 0), # service1
    # (0, 2, 0), # service1
]

for s, d, t in fixed_shift:
    model.Add(shifts[(s, d, t)] == 1)

# Fixed non-shift
# (staff, day, shift)
fixed_non_shift = [
    (0, 0, 1), # service1+
    (0, 1, 1), # service1+
    (0, 2, 1), # service1+
]

for s, d, t in fixed_non_shift:
    model.Add(shifts[(s, d, t)] == 0)


# ============================= Objectives ============================= #
obj_int_vars = []
obj_int_coeffs = []
obj_bool_vars = []
obj_bool_coeffs = []


# Regular shift
# (staff, day, shift, penalty)
requests = [
    (0, 0, 0, 1), # service1
]

for s, d, t, p in requests:
    obj_bool_vars.append(shifts[(s, d, t)])
    obj_bool_coeffs.append(p)



obj_bool_penalties = sum([coeff * variables for coeff, variables in zip(obj_bool_coeffs, obj_bool_vars)])
obj_int_penalties = sum([coeff * variables for coeff, variables in zip(obj_int_coeffs, obj_int_vars)])
model.Minimize(obj_bool_penalties + obj_int_penalties)


# ============================= Solve ============================= #


# Solution printer.
class ShiftSolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, shifts, num_staffs, num_days, num_shifts):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_staffs = num_staffs
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solutions = []

    def OnSolutionCallback(self):
        self._solution_count += 1

        #Workload per staff
        # workload = np.zeros(self._num_staffs)
        # for s in range(self._num_staffs):
        #     for d in range(self._num_days):
        #         for w in range(1, self._num_shifts): # exclude 'off'
        #             workload[s] += self.Value(self._shifts[(s, d, w)])
        # print(workload)


        # Count shift per staff
        shift_count = np.zeros((self._num_staffs, self._num_shifts))
        for s in range(self._num_staffs):
            for d in range(self._num_days):
                for t in range(self._num_shifts):
                    shift_count[s, t] += self.Value(self._shifts[(s, d, t)])
        print(shift_count)


        # w_ls = []
        # for s in range(self._num_staffs):
        #     w_ls_shift = []
        #     for d in range(self._num_days):
        #         for t in range(1, self._num_shifts):

                




        ls=[]
        for d in range(self._num_days):
            ls_day=[]
            for w in range(self._num_shifts):
                ls_shift=[]
                for s in range(self._num_staffs):
                    if self.Value(self._shifts[(s, d, w)]):
                        ls_shift.append(staff_names[s])
                ls_day.append(ls_shift)
            ls.append(ls_day)
        
        df = pd.DataFrame(ls, index=date_list, columns=shift_names)
        print(df)

        df.to_csv('schedule.csv', index=True, header=True)
        self._solutions.append(df)

    def SolutionCount(self):
        return self._solution_count

    def get_solutions(self):
        return self._solutions

    def get_solution(self):
        return self._solutions.pop()



# Solve model.
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10.0
solution_printer = ShiftSolutionPrinter(shifts, num_staffs, num_days, num_shifts)
status = solver.Solve(model, solution_printer)
# status = solver.Solve(model)
print(solver.ResponseStats())



[[ 1.  5.  3.  2.  1.  1.  3.  3. 17. 19. 12.]
 [ 1.  1.  1.  4.  3.  2.  2.  2. 20. 19. 15.]
 [ 0.  1.  4.  1.  5.  0.  3.  2. 20. 16. 15.]
 [ 2.  1.  5.  0.  2.  3.  3.  2. 18. 19. 13.]
 [ 3.  3.  3.  0.  2.  1.  2.  4. 16. 20. 13.]
 [ 0.  1.  0.  2.  3.  5.  4.  2. 16. 20. 14.]
 [ 8.  2.  0.  2.  0.  1.  2.  2. 16. 25. 14.]
 [ 1.  3.  2.  6.  0.  0.  2.  4. 21. 17. 13.]
 [ 2.  1.  2.  2.  2.  2.  5.  0. 19. 20. 15.]
 [ 3.  4.  1.  2.  3.  2.  3.  3. 13. 19. 10.]
 [ 1.  0.  1.  1.  1.  0.  2.  7. 20. 19. 18.]]
                  service1       service1+        service2       service2+  \
2023-03-01      [อ.ชานนท์]      [อ.ปริญญา]    [อ.บริบูรณ์]  [อ.ณัฐฐิกานต์]   
2023-03-02       [อ.ธีรพล]    [อ.บุญฤทธิ์]  [อ.ณัฐฐิกานต์]         [อ.บวร]   
2023-03-03       [อ.กอสิน]         [อ.บวร]  [อ.กรองกาญจน์]    [อ.บริบูรณ์]   
2023-03-04              []              []              []              []   
2023-03-05              []              []              []              []   
2023-03-06    

In [125]:
# assign to off_morning shift if Service1 & Service1+ & Observe & Morn Con & EMS & AMD is not assigned
# assign to off_afternoon shift if Service2 & Service2+ & Observe & EMS & AMD is not assigned
for s in range(num_staffs):
    for d in range(num_days):
        model.Add(sum([shifts[(s, d, t)] for t in [1, 3, 5, 6, 7, 8]])==0).OnlyEnforceIf(shifts[(s, d, 9)])
        model.Add(sum([shifts[(s, d, t)] for t in [2, 4, 5, 7, 8]])==0).OnlyEnforceIf(shifts[(s, d, 10)])
            # model.Add(shifts[(s, d, 9)] == 1).OnlyEnforceIf(shifts[(s, d, t)])



In [51]:
# ============================= Output ============================= #

solution_printer.get_solution()

,off,service1,service2,service1+,service2+,Observe,Morn Con,EMS,AMD
2023-03-01,"[อ.บวร, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ภาวิตา],[อ.บริบูรณ์],[อ.ปริญญา],[อ.ณัฐฐิกานต์],[อ.ธีรพล],[],[อ.พิมพ์พรรณ],[อ.กรองกาญจน์]
2023-03-02,"[อ.บวร, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ธีรพล],[อ.บริบูรณ์],[อ.พิมพ์พรรณ],[อ.ภาวิตา],[อ.ปริญญา],[],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์]
2023-03-03,"[อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน, อ.พิมพ์พรรณ]",[อ.ธีรพล],[อ.ภาวิตา],[อ.ณัฐฐิกานต์],[อ.ปริญญา],[อ.กรองกาญจน์],[],[อ.บวร],[อ.บริบูรณ์]
2023-03-04,"[อ.บริบูรณ์, อ.ณัฐฐิกานต์, อ.ภาวิตา, อ.ธีรพล, ...",[],[],[],[],[],[],[อ.ปริญญา],[อ.กรองกาญจน์]
2023-03-05,"[อ.บริบูรณ์, อ.ณัฐฐิกานต์, อ.ปริญญา, อ.ภาวิตา,...",[],[],[],[],[],[],[อ.พิมพ์พรรณ],[อ.กรองกาญจน์]
2023-03-06,"[อ.ณัฐฐิกานต์, อ.ปริญญา, อ.ภาวิตา, อ.ธีรพล, อ....",[],[],[],[],[],[],[อ.กรองกาญจน์],[อ.บริบูรณ์]
2023-03-07,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.พิมพ์พรรณ],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์],[อ.ธีรพล],[อ.ปริญญา],[],[อ.บริบูรณ์],[อ.บวร]
2023-03-08,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.พิมพ์พรรณ],[อ.ณัฐฐิกานต์],[อ.กรองกาญจน์],[อ.ธีรพล],[อ.บวร],[],[อ.ปริญญา],[อ.บริบูรณ์]
2023-03-09,"[อ.ภาวิตา, อ.ชานนท์, อ.บุญฤทธิ์, อ.กอสิน]",[อ.ธีรพล],[อ.กรองกาญจน์],[อ.พิมพ์พรรณ],[อ.บวร],[อ.ณัฐฐิกานต์],[],[อ.บริบูรณ์],[อ.ปริญญา]
2023-03-10,"[อ.ภาวิตา, อ.ธีรพล, อ.ชานนท์, อ.บุญฤทธิ์]",[อ.บวร],[อ.พิมพ์พรรณ],[อ.บริบูรณ์],[อ.ปริญญา],[อ.กรองกาญจน์],[],[อ.กอสิน],[อ.ณัฐฐิกานต์]
